# 01 - Data Collection & Database Sanity Check

This notebook will:
- Test downloading S&P500 OHLCV data with Alpaca API
- Save raw CSVs to disk
- Upsert them into the database
- Inspect and validate the data


In [1]:
# SETUP: Imports & Paths ===========================
from jupyter import add_project_root_to_path

import os
import pandas as pd


from src.data.collector import load_sp500_tickers, download_ticker_alpaca
from src.data.utils import upsert_ohlcv_from_df, fetch_ohlcv_to_df
from src.data.schema import OHLCV, Base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# System variables (adjust as needed)
OHLCV_DIR = os.getenv("OHLCV_DIR", "./data/ohlcv/")  # fallback if env not set
if not os.path.exists(OHLCV_DIR):
    os.makedirs(OHLCV_DIR)

DB_PATH = "sqlite:///rl_trading.db"


C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# TEST 1: Download a Few S&P 500 Tickers ===========================

tickers = load_sp500_tickers()[:3]  # Just the first 3 tickers for testing
START_DATE = "2024-01-01"
END_DATE = "2024-06-01"

for ticker in tickers:
    print(f"Downloading: {ticker}")
    df = download_ticker_alpaca(ticker, START_DATE, END_DATE)
    if not df.empty:
        print(f"Downloaded {len(df)} rows for {ticker}")
        display(df.head())
    else:
        print(f"No data for {ticker}")


Downloading: MMM
Downloaded 105 rows for MMM


,timestamp,open,high,low,close,volume,trade_count,vwap,ticker
symbol,,,,,,,,,
MMM,2024-01-02 05:00:00+00:00,108.62,110.66,108.450,110.00,2776824.0,49079.0,109.972792,MMM
MMM,2024-01-03 05:00:00+00:00,109.23,109.46,106.800,107.79,2966239.0,44742.0,107.889257,MMM
MMM,2024-01-04 05:00:00+00:00,108.08,109.34,107.705,108.17,2775925.0,37761.0,108.530158,MMM
MMM,2024-01-05 05:00:00+00:00,107.98,109.49,107.550,108.59,1665358.0,30844.0,108.578325,MMM
MMM,2024-01-08 05:00:00+00:00,108.26,108.96,107.590,108.86,2119575.0,36202.0,108.610568,MMM


Downloading: AOS
Downloaded 105 rows for AOS


,timestamp,open,high,low,close,volume,trade_count,vwap,ticker
symbol,,,,,,,,,
AOS,2024-01-02 05:00:00+00:00,81.70,82.630,81.190,81.44,613828.0,13975.0,81.561399,AOS
AOS,2024-01-03 05:00:00+00:00,80.87,80.870,78.090,78.71,1276849.0,20035.0,78.959200,AOS
AOS,2024-01-04 05:00:00+00:00,78.72,80.460,78.330,79.38,880609.0,13481.0,79.551923,AOS
AOS,2024-01-05 05:00:00+00:00,79.10,79.670,78.710,79.04,650295.0,10478.0,79.065166,AOS
AOS,2024-01-08 05:00:00+00:00,79.27,79.495,78.445,79.27,790033.0,13938.0,79.147033,AOS


Downloading: ABT
Downloaded 105 rows for ABT


,timestamp,open,high,low,close,volume,trade_count,vwap,ticker
symbol,,,,,,,,,
ABT,2024-01-02 05:00:00+00:00,109.56,111.00,109.56,109.85,5058688.0,60249.0,110.123089,ABT
ABT,2024-01-03 05:00:00+00:00,110.14,110.25,109.29,109.52,4239623.0,48984.0,109.612867,ABT
ABT,2024-01-04 05:00:00+00:00,109.68,111.03,109.51,110.98,5296139.0,52672.0,110.813969,ABT
ABT,2024-01-05 05:00:00+00:00,110.71,111.05,110.03,110.80,4127750.0,47303.0,110.698879,ABT
ABT,2024-01-08 05:00:00+00:00,111.13,112.52,110.92,112.40,5030376.0,52854.0,111.936134,ABT
